In [33]:
import pickle
import pandas as pd

In [34]:
df = pd.read_csv("../data/test.csv")
df.head(2)

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.32,Ideal,I,SI1,60.5,58.0,4.43,4.49,2.70
1,1,1.24,Premium,I,SI1,62.9,60.0,6.80,6.74,4.26


In [52]:
# Encoding 
# ==============================================================================

with open('../data/clarity_ord_encoder.pkl', 'rb') as clar_enc:
    clar_enc = pickle.load(clar_enc)  

with open('../data/color_ord_encoder.pkl', 'rb') as col_enc:
    col_enc = pickle.load(col_enc) 

with open('../data/cut_ord_encoder.pkl', 'rb') as cut_enc:
    cut_enc = pickle.load(cut_enc) 

# Estandarización
# ==============================================================================

with open('../data/rob_sacler.pkl', 'rb') as rob_scal:
    rob_scal = pickle.load(rob_scal)  

# Cargamos el mejor modelo
# ==============================================================================

with open('../data/best_modelo_dtI.pkl', 'rb') as modelo:
    modelo= pickle.load(modelo)

with open('../data/best_modelo_GB.pkl', 'rb') as modelo2:
    modelo2 = pickle.load(modelo2)

In [42]:
# Hacemos el encoding del resto de las variables
# ==============================================================================

clarity = pd.DataFrame(clar_enc.transform(df[['clarity']]), columns= ['clarity_oe'])
color = pd.DataFrame(col_enc.transform(df[['color']]), columns= ['color_oe'])
cut = pd.DataFrame(cut_enc.transform(df[['cut']]), columns= ['cut_oe'])
rob = pd.DataFrame(rob_scal.transform(df[['carat', 'depth', 'table', 'x', 'y', 'z']]), columns= ['carat', 'depth', 'table', 'x', 'y', 'z'])

In [43]:
# Unimos todos los conjuntos de datos que hemos ido generando con los transformers
# ==============================================================================

df_final = pd.concat([clarity, color, cut, rob], axis = 1)
df_final.head(1)

,clarity_oe,color_oe,cut_oe,carat,depth,table,x,y,z
0,7,1,4,0.32,60.5,58.0,4.43,4.49,2.7


In [47]:
# Cambiamos el orden de las columnas ya que estas deben estar en el mismo orden de como entrenamos al modelo
# ==============================================================================

df_final = df_final.reindex(['carat', 'depth', 'table', 'x', 'y', 'z', 'cut_oe', 'color_oe',
       'clarity_oe'], axis=1)
df_final.head(1)

,carat,depth,table,x,y,z,cut_oe,color_oe,clarity_oe
0,0.32,60.5,58.0,4.43,4.49,2.7,4,1,7


In [54]:
# Hacemos la predicción 
# ==============================================================================

df["price"] = modelo2.predict(df_final)

In [62]:
# Guardamos la submission 
# ==============================================================================

submission = pd.DataFrame(df["price"]).reset_index()
submission = submission.rename(columns={'index': 'id'})
submission.to_csv("../data/submission.csv", index=False)